# Importation


In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
pd.set_option('display.max_colwidth', None)

import warnings
warnings.filterwarnings("ignore")

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/data-assistants-with-gemma/submission_categories.txt
/kaggle/input/data-assistants-with-gemma/submission_instructions.txt
/kaggle/input/gemma/keras/gemma_2b_en/2/config.json
/kaggle/input/gemma/keras/gemma_2b_en/2/tokenizer.json
/kaggle/input/gemma/keras/gemma_2b_en/2/metadata.json
/kaggle/input/gemma/keras/gemma_2b_en/2/model.weights.h5
/kaggle/input/gemma/keras/gemma_2b_en/2/assets/tokenizer/vocabulary.spm
/kaggle/input/1000-data-science-concepts/data_science_concepts.csv
/kaggle/input/understanding-contextual-questions-answers/train.csv


# Installation of Keras


In [2]:
# Install Keras 3 last. 
!pip install -q -U keras-nlp
!pip install -q -U keras>=3

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 3.3.3 which is incompatible.


In [3]:
import sklearn
import pandas as pd
import numpy as np
from wordcloud import WordCloud, STOPWORDS
import keras
import keras_nlp
from IPython.display import display, Markdown
import matplotlib.pyplot as plt
from keras_nlp.models import GemmaCausalLM
import re
import warnings
warnings.filterwarnings('ignore')

2024-04-30 01:35:34.274992: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-30 01:35:34.275093: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-30 01:35:34.394591: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Importing OS


In [4]:
import os
os.environ["KERAS_BACKEND"] = "jax"  # Or "torch" or "tensorflow".
# Avoid memory fragmentation on JAX backend.
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"]="1.00"

## Loading Dataset
Dataset Link: https://www.kaggle.com/datasets/hserdaraltan/1000-data-science-concepts

This dataset covers more than 1000 common data science concepts.

In [5]:
data = pd.read_csv('/kaggle/input/1000-data-science-concepts/data_science_concepts.csv',nrows=200)
data.head() # First 5 rows of the dataset

,Question,Answer
0,What is under-fitting and overfitting in machine learning?,"Underfitting is when a model is too simple, and overfitting is when it's too complex, making it perform poorly on new data."
1,Can you explain what a false positive and a false negative are?,"A false positive incorrectly indicates a condition is present when it's not, while a false negative misses detecting a condition that is there."
2,Clarify the concept of Phase IV.,"Phase IV studies, also known as post-marketing surveillance, are conducted after a drug or medical product is made available to the general public. They aim to monitor the product's safety, efficacy, and long-term effects in a larger and more diverse population, providing valuable insights into real-world usage. Phase IV studies help regulators, healthcare providers, and patients make informed decisions about the product's continued use by assessing its risks and benefits over an extended period outside the controlled environment of clinical trials."
3,What is semi-supervised learning described in a short description?,"Semi-supervised learning integrates both labeled and unlabeled data during model training. By leveraging the abundance of unlabeled data alongside limited labeled data, it enhances model performance and generalization to new examples, offering scalability and efficiency in scenarios where acquiring labeled data is resource-intensive or impractical. This approach bridges the gap between supervised and unsupervised learning, unlocking the potential of vast unlabeled datasets for training robust machine learning models."
4,Discuss the parallelization of training in gradient boosting models.,"Parallelizing training of a gradient boosting model is indeed possible, leveraging the parallel processing capabilities of modern hardware, such as GPUs. Frameworks like XGBoost offer options like 'tree_method = 'gpu_hist'' to utilize GPUs for faster training. By distributing computation across multiple cores or devices simultaneously, parallelization accelerates the training process, significantly reducing training time and improving efficiency. This approach is particularly beneficial for large datasets and complex models, where traditional sequential training may be computationally intensive and time-consuming."


In [6]:
data.tail() # Last 5 rows of the dataset

,Question,Answer
195,Why are activation functions required in neural networks?,"Activation functions introduce nonlinearity, enabling neural networks to learn complex relationships between inputs and outputs, enhancing model capacity and expressiveness."
196,Can you explain a bidirectional search algorithm?,"A bidirectional search algorithm runs two simultaneous searches: one forward from the starting point and one backward from the goal. The aim is to meet in the middle, thus potentially finding a solution faster than a unidirectional search."
197,Do gradient descent methods always converge to similar points?,"Gradient descent methods may converge to different local optima, which depend on the starting conditions and the nature of the cost function."
198,Describe word2vec.,"Word2vec is a suite of models used to produce word embeddings, trained to predict surrounding words in a linguistic context."
199,What is the difference between a generative and discriminative model?,"Generative models learn data categories, while discriminative models learn category distinctions. Discriminative models generally outperform generative models in classification tasks."


## Gemma Model
**Gemma Model** is a collection of lightweight open-source generative AI (GenAI) models developed by Google DeepMind. 

**Inputs and Outputs**
* Input: Gemma models take in text strings, which can range from questions and prompts to longer documents that require summarization.
* Output: In response, they generate text in English, offering answers, summaries, or other forms of text-based output, tailored to the input provided.


This code snippet creates an instance of the GemmaCausalLM model and assigns it to the variable gemma_lm. It creates the model from a preset configuration named "gemma_instruct_2b_en".This preset specifies the architecture, hyperparameters, and other settings for the model.

In [7]:
#Create the model using the from_preset method
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma_2b_en")
gemma_lm.summary()

Attaching 'metadata.json' from model 'keras/gemma/keras/gemma_2b_en/2' to your Kaggle notebook...
Attaching 'metadata.json' from model 'keras/gemma/keras/gemma_2b_en/2' to your Kaggle notebook...
Attaching 'task.json' from model 'keras/gemma/keras/gemma_2b_en/2' to your Kaggle notebook...
Attaching 'config.json' from model 'keras/gemma/keras/gemma_2b_en/2' to your Kaggle notebook...
Attaching 'metadata.json' from model 'keras/gemma/keras/gemma_2b_en/2' to your Kaggle notebook...
Attaching 'metadata.json' from model 'keras/gemma/keras/gemma_2b_en/2' to your Kaggle notebook...
Attaching 'config.json' from model 'keras/gemma/keras/gemma_2b_en/2' to your Kaggle notebook...
Attaching 'config.json' from model 'keras/gemma/keras/gemma_2b_en/2' to your Kaggle notebook...
Attaching 'model.weights.h5' from model 'keras/gemma/keras/gemma_2b_en/2' to your Kaggle notebook...
Attaching 'metadata.json' from model 'keras/gemma/keras/gemma_2b_en/2' to your Kaggle notebook...
Attaching 'metadata.json' f

Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Tokenizer (type)                                   ┃                                             Vocab # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                   │                                             256,000 │
└────────────────────────────────────────────────────┴─────────────────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2048)        │   2,506,172,416 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     524,288,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,506,172,416 (9.34 GB)

 Trainable params: 2,506,172,416 (9.34 GB)

 Non-trainable params: 0 (0.00 B)

## Test Model


In [11]:
from memory_profiler import memory_usage

# Define input text
input_text = "What is supervised machine learning?"

# Function to measure memory usage
def measure_memory(func):
    mem_usage = memory_usage((func, (input_text,), {}))
    return max(mem_usage)  # Maximum memory usage during function execution

# Define the function to be executed
def generate_with_memory(input_text):
    outputs = gemma_lm.generate(input_text, max_length=64)
    return outputs

gemma_lm.generate("What is supervised machine learning?", max_length=64)


'What is supervised machine learning?\n\nSupervised machine learning is a type of machine learning where the algorithm is trained on a set of labeled data. The algorithm is then used to predict the outcome of new data.\n\nSupervised machine learning is a type of machine learning where the algorithm is trained on a set of labeled'